In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
import tensorflow as tf
from tensorflow import keras

In [2]:
train_df = pd.read_csv('train.csv')
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
train_df['Survived'].value_counts()

0    549
1    342
Name: Survived, dtype: int64

In [4]:
train_df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [5]:
nan_columns = train_df.columns[train_df.isnull().any()].tolist()
nan_columns

['Age', 'Cabin', 'Embarked']

In [6]:
for column in nan_columns:    
    print(f"% of null values in {column}: "
          f"{round(train_df[column].isnull().sum()/len(train_df[column])*100, 3)}")

% of null values in Age: 19.865
% of null values in Cabin: 77.104
% of null values in Embarked: 0.224


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_df.drop(columns='Survived'), 
                                                    train_df['Survived'],
                                                    test_size=0.3,
                                                    random_state=1234)

In [8]:
from sklearn.model_selection import StratifiedKFold

seed=1234
kfold = StratifiedKFold(n_splits=5)

In [9]:
sex_dict = {'female': 1, 'male': 0}

In [10]:
X_train_filtered = X_train[['Pclass', 'Fare', 'Age', 'SibSp', 'Parch', 'Sex']]
X_train_filtered = X_train_filtered.replace(sex_dict)

In [11]:
from sklearn.impute import SimpleImputer

imp = SimpleImputer(missing_values=np.nan, strategy='mean')
X_train_filtered = imp.fit_transform(X_train_filtered)

In [12]:
X_test_filtered = X_test[['Pclass', 'Fare', 'Age', 'SibSp', 'Parch', 'Sex']]
X_test_filtered = X_test_filtered.replace(sex_dict)
X_test_filtered = imp.fit_transform(X_test_filtered)

In [16]:
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_predict
from sklearn import  metrics


models = []

pipe = Pipeline([('preprocessing', StandardScaler()), ('classifier', SVC())])

param_grid = {
            'preprocessing': [StandardScaler(), None],
            'classifier__gamma': [0.001, 0.01, 0.1, 1, 10, 100],
            'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100]
}

grid_1 = GridSearchCV(pipe, param_grid, cv=kfold, return_train_score=True)

grid_1.fit(X_train_filtered, y_train)
grid_1.best_params_

y_test_pred = cross_val_predict(grid_1.best_estimator_, X_test_filtered, y_test, cv=3)
print(f"R^2: {metrics.precision_score(y_test, y_test_pred)}") 
print(f"Recall_score: {metrics.recall_score(y_test, y_test_pred)}") 

R^2: 0.7976190476190477
Recall_score: 0.6568627450980392


In [17]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(random_state=42)
log_reg.fit(X_train_filtered, y_train)
models.append(log_reg)

In [19]:
y_test_pred = cross_val_predict(log_reg, X_test_filtered, y_test, cv=3)
print(f"R^2: {metrics.precision_score(y_test, y_test_pred)}") 
print(f"Recall_score: {metrics.recall_score(y_test, y_test_pred)}") 

R^2: 0.813953488372093
Recall_score: 0.6862745098039216


In [20]:
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(n_estimators=1000)
random_forest.fit(X_train_filtered, y_train)

Y_prediction = random_forest.predict(X_test_filtered)

print("R^2: {}".format(metrics.precision_score(y_test, random_forest.predict(X_test_filtered)) ))
print("recall_score: {}".format( metrics.recall_score(y_test, random_forest.predict(X_test_filtered)) ))

R^2: 0.7701149425287356
recall_score: 0.6568627450980392


In [21]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=10), 
    n_estimators=1, learning_rate=0.5, 
    algorithm="SAMME.R", random_state=42)
ada_clf.fit(X_train_filtered, y_train)
print("R^2: {}".format(metrics.precision_score(y_test, ada_clf.predict(X_test_filtered)) ))
print("recall_score: {}".format( metrics.recall_score(y_test, ada_clf.predict(X_test_filtered)) ))

R^2: 0.7948717948717948
recall_score: 0.6078431372549019


In [28]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import History

history = History()
model = keras.models.Sequential([
    keras.layers.Dense(300, activation="relu", input_shape=(X_train_filtered.shape[1],)),
    keras.layers.Dense(200, activation="swish"),
    keras.layers.Dense(100, activation="elu"),
    keras.layers.Dense(50, activation="elu"),
    keras.layers.Dense(1, activation="sigmoid")
])

model.compile(loss="binary_crossentropy", optimizer="Adam", metrics=["accuracy"])
history = model.fit(X_train_filtered, y_train, validation_data= (X_test_filtered, y_test), epochs=100)

Epoch 1/100
20/20 [==============================] - 1s 11ms/step - loss: 0.8693 - accuracy: 0.6501 - val_loss: 0.7158 - val_accuracy: 0.6157
Epoch 2/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6834 - accuracy: 0.6693 - val_loss: 0.6543 - val_accuracy: 0.6418
Epoch 3/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5910 - accuracy: 0.7079 - val_loss: 0.6210 - val_accuracy: 0.6418
Epoch 4/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5826 - accuracy: 0.7239 - val_loss: 0.6443 - val_accuracy: 0.6791
Epoch 5/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5636 - accuracy: 0.7127 - val_loss: 0.6967 - val_accuracy: 0.6493
Epoch 6/100
20/20 [==============================] - 0s 5ms/step - loss: 0.5761 - accuracy: 0.7287 - val_loss: 0.7046 - val_accuracy: 0.6194
Epoch 7/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5577 - accuracy: 0.7303 - val_loss: 0.6266 - val_accuracy: 0.6642
Epoch 8/100


Ostatecznie wybrałam model na sieci z kerasa.

In [29]:
test_df = pd.read_csv("test.csv")
test = test_df[['Pclass', 'Fare', 'Age', 'SibSp', 'Parch', 'Sex']]
test = test.replace(sex_dict)
test = imp.fit_transform(test)

In [30]:
y_prob = model.predict(test) 

14/14 [==============================] - 0s 1ms/step


In [31]:
y_classes = abs(y_prob.round()).astype('int').squeeze()

In [32]:
output = pd.DataFrame({'PassengerId': test_df['PassengerId'], 'Survived': y_classes})

In [33]:
output.to_csv('results.csv', index=False)